In [1]:
import pysam

In [2]:
bam = pysam.AlignmentFile('data/Deduped-1057_0.bam')

In [13]:
bam.check_index()

True

In [14]:
bam.get_index_statistics()

[IndexStats(contig='chr1', mapped=1129995, unmapped=0, total=1129995),
 IndexStats(contig='chr10', mapped=1072466, unmapped=0, total=1072466),
 IndexStats(contig='chr11', mapped=71491, unmapped=0, total=71491),
 IndexStats(contig='chr11_KI270721v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr12', mapped=234431, unmapped=0, total=234431),
 IndexStats(contig='chr13', mapped=1604875, unmapped=0, total=1604875),
 IndexStats(contig='chr14', mapped=61808, unmapped=0, total=61808),
 IndexStats(contig='chr14_GL000009v2_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_GL000225v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_KI270722v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_GL000194v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_KI270723v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_KI270724v1_random', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14_KI27072

In [35]:
a = next(bam.fetch(parser=pysam.asGTF()))

TypeError: fetch() got an unexpected keyword argument 'parser'

In [34]:
dir(pysam.asGTF(a).get_encoding())

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'aend',
 'alen',
 'aligned_pairs',
 'bin',
 'blocks',
 'cigar',
 'cigarstring',
 'cigartuples',
 'compare',
 'flag',
 'from_dict',
 'fromstring',
 'get_aligned_pairs',
 'get_blocks',
 'get_cigar_stats',
 'get_forward_qualities',
 'get_forward_sequence',
 'get_overlap',
 'get_reference_positions',
 'get_reference_sequence',
 'get_tag',
 'get_tags',
 'has_tag',
 'header',
 'infer_query_length',
 'infer_read_length',
 'inferred_length',
 'is_duplicate',
 'is_paired',
 'is_proper_pair',
 'is_qcfail',
 'is_read1',
 'is_read2',
 'is_reverse',
 'is_secondary',
 'is_supplementary',
 'is_unmapped',
 'isize',
